<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/yolo/Algorithums/detection/Yolo/YOLO_FLOWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Flower

## Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
basicdata={'account':'sanashajeeva@gmail.com','user':"Yuka"}


In [3]:
! locale-gen en_US.UTF-8
! export LC_ALL=en_US.UTF-8

Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


## Intallations

In [4]:
!pip install firebase-admin

In [5]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.7/871.7 kB 13.6 MB/s eta 0:00:00


## Imports

In [6]:
import firebase_admin


In [7]:
from firebase_admin import credentials,storage


In [8]:
from firebase_admin import firestore

In [9]:
from ultralytics import YOLO

In [10]:
from datetime import date,datetime

In [11]:
import os

In [12]:
import numpy as np

In [13]:
from matplotlib import pyplot as plt

In [14]:
import random

In [15]:
import yaml

In [16]:
import csv

In [17]:
import json

## Config

In [18]:
epochs=10
imageSize=640
optimizer='Adam'
dropout=0.1

### Dirctory

In [19]:
work_dirctory="/content/drive/MyDrive/AyushVision/yolo8/training"
work_dirctory_root="/content/drive/MyDrive/AyushVision/yolo8"
firebase_key="/content/drive/MyDrive/AyushVision/ServiceKey.json"

In [20]:
today_work_dirctory=f"{work_dirctory}/{date.today().strftime('%d_%m_%Y')}/{date.today().strftime('%H%M%S')}"

In [21]:
if not os.path.exists(work_dirctory):
  os.makedirs(work_dirctory,exist_ok=True)
if not os.path.exists(today_work_dirctory):
  os.makedirs(today_work_dirctory,exist_ok=True)

In [22]:
data="/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml"

In [23]:
project=f"{today_work_dirctory}/flowers/runs"
name=f'train{epochs}'

## Firebase

In [24]:
firebase_count=1
if firebase_count==1:
  cred = credentials.Certificate(firebase_key)
  firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
  })
firebase_count=firebase_count+1

In [25]:
db = firestore.client()

In [26]:
bucket = storage.bucket()

In [27]:
yolo_ref = db.collection("training1").document('yolo')
doc = yolo_ref.get()

In [28]:
if not doc.exists:
  doc_ref= db.collection("training1").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'V8','mode':'detection','url':'ultralytics'})
else:
  doc_ref= yolo_ref

### Roboflow

## Model

In [29]:
today_ref=doc_ref.collection("Flower").document(datetime.now().strftime(f"%Y%m%d%H"))

In [30]:
basicdata['imageSize']=imageSize
basicdata['project']=project
basicdata['name']=name
basicdata['training_date']=datetime.now().strftime("%Y/%m/%d %H:%M:%S")
basicdata['filename']='YOLO_FLOWER'

In [31]:
with open(data, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [32]:
basicdata['names']=yaml_content['names']

In [33]:
today_ref.set(basicdata)

update_time {
  seconds: 1725582992
  nanos: 465828000
}

In [34]:
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 22.8MB/s]


Transferred 355/355 items from pretrained weights


#### Training

In [35]:
results = yolo_model.train(data=data,name=name, epochs=epochs, imgsz=imageSize,project=project,optimizer=optimizer,plots=True,dropout=dropout)

Ultralytics YOLOv8.2.88 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/AyushVision/yolo8/training/06_09_2024/000000/flowers/runs, name=train10, exist_ok=False, pretrained=yolov8n.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

100%|██████████| 755k/755k [00:00<00:00, 4.37MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/train/labels.cache... 4620 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4620/4620 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/valid/labels.cache... 441 images, 1 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/AyushVision/yolo8/training/06_09_2024/000000/flowers/runs/train10/labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/AyushVision/yolo8/training/06_09_2024/000000/flowers/runs/train10
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      2.061      3.287      2.423         13        640: 100%|██████████| 289/289 [1:06:08<00:00, 13.73s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  79%|███████▊  | 11/14 [05:26<01:28, 29.65s/it]


KeyboardInterrupt: 

In [ ]:
args_yaml=f'{project}/{name}/args.yaml'

In [ ]:
resultcsv=f'{project}/{name}/results.csv'

In [ ]:
with open(resultcsv, mode='r') as file:
     csv_reader = csv.DictReader(file)
     data_dict = [row for row in csv_reader]

In [ ]:
with open(args_yaml, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [ ]:
today_training_colref=today_ref.collection('Train')

In [ ]:
today_training_ref=today_training_colref.document('args')

In [ ]:
today_training_ref.set(yaml_content)

In [ ]:
todayresultref=today_training_colref.document('results')

In [ ]:
todayresultref.set({'results':data_dict})

In [ ]:
trainfiles={}
wights={}

In [ ]:
for file in os.listdir(f"{project}/{name}"):
  if os.path.isfile(f"{project}/{name}/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/{name}/{file}")
    blob.make_public()
    trainfiles[os.path.splitext(file)[0]]=blob.public_url


In [ ]:
for file in os.listdir(f"{project}/{name}/weights"):
  if os.path.isfile(f"{project}/{name}/weights/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/weights/{file}")
    blob.upload_from_filename(f"{project}/{name}/weights/{file}")
    blob.make_public()
    wights[os.path.splitext(file)[0]]=blob.public_url

In [ ]:
today_training_files=today_training_colref.document('files')
today_training_files.set(trainfiles)

In [ ]:
today_weights_files=today_training_colref.document('weights')
today_weights_files.set(wights)

#### Validation

In [ ]:
model_train_path=f'{project}/{name}/weights/best.pt'

In [ ]:
model = YOLO(model_train_path)  # load a custom trained model

In [ ]:
metrics = model.val(save_json=True,plots=True,project=project,name=f'val{epochs}')  # no arguments needed, dataset and settings remembered

In [ ]:
val_today_col=today_ref.collection('Val')

In [ ]:
predication_file=f'{project}/val{epochs}/predictions.json'

In [ ]:
with open(predication_file, 'r') as file:
    data_dict = json.load(file)

In [ ]:
val_today_ref_files=val_today_col.document('files')

In [ ]:
valfiles={}

In [ ]:
for file in os.listdir(f"{project}/val{epochs}"):
  if os.path.isfile(f"{project}/val{epochs}/{file}"):
    blob = bucket.blob(f"yolo/flower/val/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/val{epochs}/{file}")
    blob.make_public()
    valfiles[os.path.splitext(file)[0]]=blob.public_url

In [ ]:
val_today_ref_files.set(valfiles)

In [ ]:
val_today_ref_metrics=val_today_col.document('metrics')

In [ ]:

val_today_ref_metrics.set({
    'confusion_matrix':{
        'conf':metrics.confusion_matrix.conf
    },
    'maps':np.array(metrics.maps).tolist(),
    'fitness':metrics.fitness,
'results_dict':metrics.results_dict,
    'speed':metrics.speed,
    'box':{
        'map':metrics.box.map,
        'map50':metrics.box.map50,
        'map75':metrics.box.map75,
        'mp':metrics.box.mp,
'mr':metrics.box.mr,
        'nc':metrics.box.nc,
        'p':np.array(metrics.box.p).tolist(),
        'r':np.array(metrics.box.r).tolist(),
        'ap':np.array(metrics.box.ap).tolist(),
        'ap50':np.array(metrics.box.ap50).tolist(),
        'ap_class_index':np.array(metrics.box.ap_class_index).tolist(),
        }
})

In [ ]:
metrics.box.map50

In [ ]:
metrics.box.map75

In [ ]:
metrics.box.maps

#### Predict

In [ ]:

results=model.predict(source=f'{work_dirctory_root}/dataset/ayushvision_flowers/test/images',save=True)

In [ ]:
predicts=[]

In [ ]:
for file in os.listdir(results[0].save_dir):
  if os.path.isfile(f"{results[0].save_dir}/{file}"):
    blob = bucket.blob(f"yolo/flower/predict/{epochs}/{file}")
    blob.upload_from_filename(f"{results[0].save_dir}/{file}")
    blob.make_public()
    predicts.append(blob.public_url)

In [ ]:
today_ref.collection('Predict').add({
    'files':predicts
})

### Export Model

In [ ]:
format='onnx'
model.export(format=format,keras=True,dynamic=True,simplify=True,)

In [ ]:
blob = bucket.blob(f"yolo/flower/model/{epochs}/best.onnx")
blob.upload_from_filename(f"{today_work_dirctory}/flowers/runs/{name}/weights/best.onnx")
blob.make_public()

In [ ]:
today_ref.collection('Model').add({
    'format':format,
    "keras":True,
    "dynamic":True,
    "simplify":True,
    "model":blob.public_url
})